In [1]:
import numpy as np
import scipy.misc
from scipy.fftpack import dct, idct
import sys
from PIL import Image
import matplotlib
import matplotlib.pyplot as plt
import random
from tqdm._tqdm_notebook import tqdm_notebook
from scipy.fftpack import dct, idct
import seaborn as sns
from skimage.metrics import structural_similarity as ssim
import pandas as pd
import sympy

%matplotlib inline


class ImageLoader:
    def __init__(self, FILE_PATH):
        self.img = np.array(Image.open(FILE_PATH))
        # 行数
        self.row_blocks_count = self.img.shape[0] // 8
        # 列数
        self.col_blocks_count = self.img.shape[1] // 8

    def get_points(self, POINT):
        Row = random.randint(0, len(self.img) - POINT - 1)
        Col = random.randint(0, len(self.img) - 1)
        return self.img[Row : Row + POINT, Col]

    def get_block(self, col, row):
        return self.img[col * 8 : (col + 1) * 8, row * 8 : (row + 1) * 8]


# plt.rcParams['font.family'] ='sans-serif'#使用するフォント
# plt.rcParams["font.sans-serif"] = "Source Han Sans"
plt.rcParams["font.family"] = "Source Han Sans JP"  # 使用するフォント
plt.rcParams["xtick.direction"] = "in"  # x軸の目盛線が内向き('in')か外向き('out')か双方向か('inout')
plt.rcParams["ytick.direction"] = "in"  # y軸の目盛線が内向き('in')か外向き('out')か双方向か('inout')
plt.rcParams["xtick.major.width"] = 1.0  # x軸主目盛り線の線幅
plt.rcParams["ytick.major.width"] = 1.0  # y軸主目盛り線の線幅
plt.rcParams["font.size"] = 12  # フォントの大きさ
plt.rcParams["axes.linewidth"] = 1.0  # 軸の線幅edge linewidth。囲みの太さ
matplotlib.font_manager._rebuild()

MONO_DIR_PATH = "../../Mono/"
AIRPLANE = ImageLoader(MONO_DIR_PATH + "airplane512.bmp")
BARBARA = ImageLoader(MONO_DIR_PATH + "barbara512.bmp")
BOAT = ImageLoader(MONO_DIR_PATH + "boat512.bmp")
GOLDHILL = ImageLoader(MONO_DIR_PATH + "goldhill512.bmp")
LENNA = ImageLoader(MONO_DIR_PATH + "lenna512.bmp")
MANDRILL = ImageLoader(MONO_DIR_PATH + "mandrill512.bmp")
MILKDROP = ImageLoader(MONO_DIR_PATH + "milkdrop512.bmp")
SAILBOAT = ImageLoader(MONO_DIR_PATH + "sailboat512.bmp")

Images = np.array(
    [
        AIRPLANE,
        BARBARA,
        BOAT,
        GOLDHILL,
        LENNA,
        MANDRILL,
        MILKDROP,
        SAILBOAT
    ]
)

In [2]:
n_bar = 5
N = 8
OUTPUT_DIR_PATH = "../../Plot_tool/output/images/DMLCT/" + str(N) + "x" + str(N) + "/"

# MSDS

In [3]:
def msds(N,arr):
    w_e = 0
    e_e = 0
    n_e = 0
    s_e = 0
    nw_e = 0
    ne_e = 0
    sw_e = 0
    se_e = 0
    for row in range(arr.shape[0] // N):
        for col in range(arr.shape[1] // N):
            f_block = arr[row * N : (row + 1) * N, col * N : (col + 1) * N]
            # w
            if col == 0:
                w_block = np.fliplr(f_block)
            else:
                w_block = arr[row * N : (row + 1) * N, (col - 1) * N : col * N]
            # e
            if col == arr.shape[1] // N - 1:
                e_block = np.fliplr(f_block)
            else:
                e_block = arr[row * N : (row + 1) * N, (col + 1) * N : (col + 2) * N]
            # n
            if row == 0:
                n_block = np.flipud(f_block)
            else:
                n_block = arr[(row - 1) * N : row * N, col * N : (col + 1) * N]
            # s
            if row == arr.shape[0] // N - 1:
                s_block = np.flipud(f_block)
            else:
                s_block = arr[(row + 1) * N : (row + 2) * N, col * N : (col + 1) * N]

            w_d1 = f_block[:, 0] - w_block[:, N-1]
            e_d1 = f_block[:, N-1] - e_block[:, 0]
            n_d1 = f_block[0, :] - n_block[N-1, :]
            s_d1 = f_block[N-1, :] - s_block[0, :]

            w_d2 = (w_block[:, N-1] - w_block[:, N-2] + f_block[:, 1] - f_block[:, 0]) / 2
            e_d2 = (e_block[:, 1] - e_block[:, 0] + f_block[:, N-1] - f_block[:, N-2]) / 2
            n_d2 = (n_block[N-1, :] - n_block[N-2, :] + f_block[1, :] - f_block[0, :]) / 2
            s_d2 = (s_block[1, :] - s_block[0, :] + f_block[N-1, :] - f_block[N-2, :]) / 2

            w_e += np.sum((w_d1 - w_d2) ** 2 )
            e_e += np.sum((e_d1 - e_d2) ** 2 )
            n_e += np.sum((n_d1 - n_d2) ** 2)
            s_e += np.sum((s_d1 - s_d2) ** 2)

            # nw
            if row == 0 or col == 0:
                nw_block = np.flipud(np.fliplr(f_block))
            else:
                nw_block = arr[(row - 1) * N : row * N, (col - 1) * N : col * N]
            # ne
            if row == 0 or col == arr.shape[1] // N - 1:
                ne_block = np.flipud(np.fliplr(f_block))
            else:
                ne_block = arr[(row-1) * N : row * N, (col + 1) * N : (col + 2) * N]
            # sw
            if row == arr.shape[0] // N -1 or col == 0:
                sw_block = np.flipud(np.fliplr(f_block))
            else:
                sw_block = arr[row * N : (row+1) * N, (col-1) * N : col * N]
            # se
            if  row == arr.shape[0]//N-1 or col == arr.shape[0] // N -1:
                se_block = np.flipud(np.fliplr(f_block))
            else:
                se_block = arr[(row + 1) * N : (row + 2) * N, (col+1) * N : (col + 2) * N]

            nw_g1 = f_block[0, 0] - nw_block[N-1, N-1]
            ne_g1 = f_block[0, N-1] - ne_block[N-1, 0]
            sw_g1 = f_block[N-1, 0] - sw_block[0, N-1]
            se_g1 = f_block[N-1, N-1] - se_block[0, 0]

            nw_g2 = (nw_block[N-1,N-1] - nw_block[N-2,N-2] + f_block[1,1] - f_block[0,0])/2
            ne_g2 = (ne_block[N-1,0] - ne_block[N-2,1] + f_block[1,N-2] - f_block[0,N-1])/2
            sw_g2 = (sw_block[0,N-1] - nw_block[1,N-2] + f_block[N-2,1] - f_block[N-1,0])/2
            se_g2 = (nw_block[0,0] - nw_block[1,1] + f_block[N-2,N-2] - f_block[N-1,N-1])/2

            nw_e += (nw_g1 - nw_g2) ** 2 
            ne_e += (ne_g1 - ne_g2) ** 2 
            sw_e += (sw_g1 - sw_g2) ** 2 
            se_e += (se_g1 - se_g2) ** 2 

    MSDSt = (w_e + e_e + n_e + s_e + nw_e + ne_e + sw_e + se_e)/ ((arr.shape[0]/N)**2)
    MSDS1 = (w_e + e_e + n_e + s_e)/ ((arr.shape[0]/N)**2)
    MSDS2 = (nw_e + ne_e + sw_e + se_e)/ ((arr.shape[0]/N)**2)
    return MSDSt, MSDS1, MSDS2

In [4]:
class DMLCT:
    def __init__(self, n_bar, N):
        self.n_bar = n_bar
        self.N = N

        self.x_l = (2 * np.arange(N) + 1) / (2 * N)
        self.s_l = np.arange(n_bar) / (n_bar - 1)
        self.xi = (np.arange(n_bar + 1) - 0.5) / (n_bar - 1)

        self.lambda_kh = self.get_lambda_kh(self.n_bar)

        self.w_k_j = self.get_w_k_j(self.n_bar, self.N)
        self.W_L_k_kh = self.get_W_L_k_kh(self.n_bar, self.N)
        self.W_k_kh = self.get_W_k_kh(self.n_bar, self.N)
        self.W_R_k_kh = self.get_W_R_k_kh(self.n_bar, self.N)

    def Lagrange_j(self, j):
        x = sympy.Symbol("x")
        L_x = 1.0
        for l in range(self.n_bar):
            if l != j:
                L_x *= (x - self.s_l[l]) / (self.s_l[j] - self.s_l[l])
        return sympy.integrate(L_x)

    def get_lambda_kh(self, n_bar):
        lambda_kh = np.ones(n_bar)
        lambda_kh[0] = np.sqrt(1 / 2)
        return lambda_kh

    def get_w_k_j(self, n_bar, N):
        L_j = np.zeros((n_bar, N))
        x = sympy.Symbol("x")
        for j in range(n_bar):
            temp = []
            Lj = self.Lagrange_j(j)
            for k in range(N):
                temp.append(Lj.subs(x, self.x_l[k]))
            L_j[j] = np.array(temp)

        w_k_j = np.zeros((n_bar, N))
        for j in range(n_bar):
            w_k_j[j] = scipy.fftpack.dct(L_j[j], norm="ortho")
        return w_k_j

    def get_W_L_k_kh(self, n_bar, N):
        W_L_k_kh = np.zeros((n_bar - 1, N))
        lambda_kh = self.get_lambda_kh(n_bar)

        for kh in range(n_bar - 1):
            W_L_k_kh[kh] = (
                (1 - n_bar)
                * np.sqrt(2 / N)
                * lambda_kh[kh]
                * np.cos(np.pi * kh * (self.xi[0] + 1))
                * self.w_k_j[0]
            )
        return W_L_k_kh

    def get_W_k_kh(self, n_bar, N):
        W_k_kh = np.zeros((n_bar - 1, N))
        for kh in range(n_bar - 1):
            sum_sin = np.zeros(N)
            for j in range(1, n_bar - 2 + 1):
                sum_sin += np.sin(np.pi * kh * self.s_l[j]) * self.w_k_j[j]

            W_k_kh[kh] = (
                (n_bar - 1)
                * np.sqrt(2 / N)
                * self.lambda_kh[kh]
                * (
                    np.cos(np.pi * kh * self.xi[1])
                    * (self.w_k_j[0] - (-1) ** (kh) * self.w_k_j[n_bar - 1])
                    - 2 * np.sin((np.pi * kh) / (2 * (n_bar - 1))) * sum_sin
                )
            )
        return W_k_kh

    def get_W_R_k_kh(self, n_bar, N):
        W_R_k_kh = np.zeros((n_bar - 1, N))
        for kh in range(n_bar - 1):
            W_R_k_kh[kh] = (
                (n_bar - 1)
                * np.sqrt(2 / N)
                * self.lambda_kh[kh]
                * np.cos(np.pi * kh * (self.xi[n_bar] - 1))
                * self.w_k_j[n_bar - 1]
            )
        return W_R_k_kh

In [5]:
def get_F_L_k_horizontal(arr, N, row, col):
    # w
    if col == 0:
#         w_block = np.zeros(N)
        w_block = arr[row, col * N : (col + 1) * N]
    else:
        w_block = arr[row, (col - 1) * N : col * N]

    return w_block

In [6]:
def get_F_R_k_horizontal(arr, N, row, col):
    # e
    if col == arr.shape[1] // N - 1:
#         e_block = np.zeros(N)
        e_block = arr[row, col * N : (col + 1) * N]
    else:
        e_block = arr[row, (col + 1) * N : (col + 2) * N]

    return e_block

In [7]:
def get_F_L_k_vertical(arr, N, row, col):
    # n
    if row == 0:
#         n_block = np.zeros(N)
        n_block = arr[row * N : (row + 1) * N, col]
    else:
        n_block = arr[(row - 1) * N : row * N, col]

    return n_block

In [8]:
def get_F_R_k_vertical(arr, N, row, col):
    # s
    if row == arr.shape[0] // N - 1:
#         s_block = np.zeros(N)
        s_block = arr[row * N : (row + 1) * N, col]
    else:
        s_block = arr[(row + 1) * N : (row + 2) * N, col]

    return s_block

# 量子化テーブル

In [9]:
Q50_Luminance = np.array(
    [
        [16, 11, 10, 16, 24, 40, 51, 61],
        [12, 12, 14, 19, 26, 58, 60, 55],
        [14, 13, 16, 24, 40, 57, 69, 56],
        [14, 17, 22, 29, 51, 87, 80, 62],
        [18, 22, 37, 56, 68, 109, 103, 77],
        [24, 35, 55, 64, 81, 104, 113, 92],
        [49, 64, 78, 87, 103, 121, 120, 101],
        [72, 92, 95, 98, 112, 100, 103, 99],
    ]
)

In [10]:
option = "n" + str(n_bar) + "_" + str(N) + "x" + str(N)
img_name = ["Airplane","Barbara","Boat","Goldhill","Lenna","Mandrill","Milkdrop","Sailboat"]
df = pd.read_excel("../../Plot_tool/xlsx/DMLCT_"+ option +".xlsx",sheet_name=None)

for i in range(len(Images)):
    name = img_name[i]
    print(name)
    IMG = Images[i]
    df_Q = df[name]["Q"]
 
    for j in range(13):
        bpp = 0.2 + 0.05 * j
        Q = df_Q[j]
        Fk = np.zeros(IMG.img.shape)
        # 順変換
        ## 縦方向
        ### DCT
        for row in range(IMG.img.shape[0] // N):
            for col in range(IMG.img.shape[1]):
                eight_points = IMG.img[N * row : N * (row + 1), col]
                c = scipy.fftpack.dct(eight_points, norm="ortho")
                Fk[N * row : N * (row + 1), col] = c
        ### 残差
        dmlct = DMLCT(n_bar, N)
        for row in range(IMG.img.shape[0] // N):
            for col in range(IMG.img.shape[1]):
                # ビューなら直接いじっちゃう
                F = Fk[N * row : N * (row + 1), col]
                F_L = get_F_L_k_vertical(Fk, N, row, col)
                F_R = get_F_R_k_vertical(Fk, N, row, col)

                U_k_n_bar = np.zeros(N)
                for kh in range(n_bar - 2 + 1):
                    U_k_n_bar += (
                        F_L[kh] * dmlct.W_L_k_kh[kh]
                        + F[kh] * dmlct.W_k_kh[kh]
                        + F_R[kh] * dmlct.W_R_k_kh[kh]
                    )

                # n_bar = 4 なら 0,1,2は残す 3,4,5,6,7を書き換える
                F[n_bar - 2 + 1 :] -= U_k_n_bar[n_bar - 2 + 1 :]
        # 0を残す
        for k in reversed(range(1, n_bar - 2 + 1)):
            dmlct = DMLCT(k+1, N)
            for row in range(IMG.img.shape[0] // N):
                for col in range(IMG.img.shape[1]):
                    # ビューなら直接いじっちゃう
                    F = Fk[N * row : N * (row + 1), col]
                    F_L = get_F_L_k_vertical(Fk, N, row, col)
                    F_R = get_F_R_k_vertical(Fk, N, row, col)

                    U_k_n_bar = np.zeros(N)

                    for kh in range((k + 1) - 2 + 1):
                        U_k_n_bar += (
                            F_L[kh] * dmlct.W_L_k_kh[kh]
                            + F[kh] * dmlct.W_k_kh[kh]
                            + F_R[kh] * dmlct.W_R_k_kh[kh]
                        )

                    F[k] -= U_k_n_bar[k]
        ## 横方向
        ### DCT
        for row in range(Fk.shape[0]):
            for col in range(Fk.shape[1] // N):
                eight_points = Fk[row, N * col : N * (col + 1)]
                c = scipy.fftpack.dct(eight_points, norm="ortho")
                Fk[row, N * col : N * (col + 1)] = c
        ### 残差
        dmlct = DMLCT(n_bar, N)
        for row in range(IMG.img.shape[0]):
            for col in range(IMG.img.shape[1] // N):
                F = Fk[row, N * col : N * (col + 1)]
                F_L = get_F_L_k_horizontal(Fk, N, row, col)
                F_R = get_F_R_k_horizontal(Fk, N, row, col)

                U_k_n_bar = np.zeros(N)
                for kh in range(n_bar - 2 + 1):
                    U_k_n_bar += (
                        F_L[kh] * dmlct.W_L_k_kh[kh]
                        + F[kh] * dmlct.W_k_kh[kh]
                        + F_R[kh] * dmlct.W_R_k_kh[kh]
                    )

                # n_bar = 4 なら 0,1,2は残す 3,4,5,6,7を書き換える
                F[n_bar - 2 + 1 :] -= U_k_n_bar[n_bar - 2 + 1 :]
        # 0を残す
        for k in reversed(range(1, n_bar - 2 + 1)):
            dmlct = DMLCT(k+1, N)
            for row in range(IMG.img.shape[0]):
                for col in range(IMG.img.shape[1] // N):
                    F = Fk[row, N * col : N * (col + 1)]
                    F_L = get_F_L_k_horizontal(Fk, N, row, col)
                    F_R = get_F_R_k_horizontal(Fk, N, row, col)

                    U_k_n_bar = np.zeros(N)

                    for kh in range((k + 1) - 2 + 1):
                        U_k_n_bar += (
                            F_L[kh] * dmlct.W_L_k_kh[kh]
                            + F[kh] * dmlct.W_k_kh[kh]
                            + F_R[kh] * dmlct.W_R_k_kh[kh]
                        )

                    F[k] -= U_k_n_bar[k]
                    
        Fk_Ori = np.copy(Fk)
        # Q=N0における量子化テーブルをスケーリング
#         Q = 12
        if Q < 50:
            S = 5000 / Q
        else:
            S = 200 - 2 * Q

        S = np.floor(S)

        Q_Luminance = np.zeros((N, N))
        for row in range(N):
            for col in range(N):
                Q_Luminance[row, col] = np.floor((S * Q50_Luminance[row, col] + 50) / 100)
        
        Fk = np.copy(Fk_Ori)
        Q_Fk = np.zeros(Fk.shape)
        for row in range(IMG.img.shape[0] // N):
            for col in range(IMG.img.shape[1] // N):
                block = Fk[row * N : (row + 1) * N, col * N : (col + 1) * N]
                # 量子化
                block = np.round(block / Q_Luminance)
                # 逆量子化
                block = block * Q_Luminance
                Q_Fk[row * N : (row+1)*N, col * N : (col+1)*N] = block
                
        Fk = np.copy(Q_Fk)
        Q_recover = np.zeros(Q_Fk.shape)
        
        # 横方向
        ## 残差
        for k in range(1, n_bar - 2 + 1):
            dmlct = DMLCT(k+1, N)
            for row in range(IMG.img.shape[0]):
                for col in range(IMG.img.shape[1] // N):
                    F = Fk[row, N * col : N * (col + 1)]
                    F_L = get_F_L_k_horizontal(Fk, N, row, col)
                    F_R = get_F_R_k_horizontal(Fk, N, row, col)

                    U_k_n_bar = np.zeros(N)
                    for kh in range((k + 1) - 2 + 1):
                        U_k_n_bar += (
                            F_L[kh] * dmlct.W_L_k_kh[kh]
                            + F[kh] * dmlct.W_k_kh[kh]
                            + F_R[kh] * dmlct.W_R_k_kh[kh]
                        )

                    F[k] += U_k_n_bar[k]
                    
        dmlct = DMLCT(n_bar, N)
        for row in range(IMG.img.shape[0]):
            for col in range(IMG.img.shape[1] // N):
                F = Fk[row, N * col : N * (col + 1)]
                F_L = get_F_L_k_horizontal(Fk, N, row, col)
                F_R = get_F_R_k_horizontal(Fk, N, row, col)

                U_k_n_bar = np.zeros(N)
                for kh in range(n_bar - 2 + 1):
                    U_k_n_bar += (
                        F_L[kh] * dmlct.W_L_k_kh[kh]
                        + F[kh] * dmlct.W_k_kh[kh]
                        + F_R[kh] * dmlct.W_R_k_kh[kh]
                    )

                # n_bar = 4 なら 0,1,2は残す 3,4,5,6,7を書き換える
                F[n_bar - 2 + 1 :] += U_k_n_bar[n_bar - 2 + 1 :]
                
        ## IDCT
        for row in range(Fk.shape[0]):
            for col in range(Fk.shape[1] // N):
                F = Fk[row, N * col : N * col + N]
                data = scipy.fftpack.idct(F, norm="ortho")
                # Fkに代入した後、縦方向に対して処理
                Fk[row, N * col : N * col + N] = data
        ## 縦方向        
        ### 残差
        for k in range(1, n_bar - 2 + 1):
            dmlct = DMLCT(k+1, N)
            for row in range(IMG.img.shape[0] // N):
                for col in range(IMG.img.shape[1]):
                    # ビューなら直接いじっちゃう
                    F = Fk[N * row : N * (row + 1), col]
                    F_L = get_F_L_k_vertical(Fk, N, row, col)
                    F_R = get_F_R_k_vertical(Fk, N, row, col)

                    U_k_n_bar = np.zeros(N)

                    for kh in range((k + 1) - 2 + 1):
                        U_k_n_bar += (
                            F_L[kh] * dmlct.W_L_k_kh[kh]
                            + F[kh] * dmlct.W_k_kh[kh]
                            + F_R[kh] * dmlct.W_R_k_kh[kh]
                        )

                    F[k] += U_k_n_bar[k]
                    
        dmlct = DMLCT(n_bar, N)
        for row in range(IMG.img.shape[0] // N):
            for col in range(IMG.img.shape[1]):
                # ビューなら直接いじっちゃう
                F = Fk[N * row : N * (row + 1), col]
                F_L = get_F_L_k_vertical(Fk, N, row, col)
                F_R = get_F_R_k_vertical(Fk, N, row, col)

                U_k_n_bar = np.zeros(N)
                for kh in range(n_bar - 2 + 1):
                    U_k_n_bar += (
                        F_L[kh] * dmlct.W_L_k_kh[kh]
                        + F[kh] * dmlct.W_k_kh[kh]
                        + F_R[kh] * dmlct.W_R_k_kh[kh]
                    )

                # n_bar = 4 なら 0,1,2は残す 3,4,5,6,7を書き換える
                F[n_bar - 2 + 1 :] += U_k_n_bar[n_bar - 2 + 1 :]
        ### IDCT
        for row in range(Fk.shape[0] // N):
            for col in range(Fk.shape[1]):
                F = Fk[N * row : N * (row + 1), col]
                data = scipy.fftpack.idct(F, norm="ortho")
                # 復元画像
                Q_recover[N * row : N * (row + 1), col] = data
                
        Q_recover = np.round(Q_recover)
        
        plt.imsave(OUTPUT_DIR_PATH + "DMLCT_" + option + "_" + name + "_" + "{:.2f}".format(bpp) + ".png",Q_recover,cmap="gray")

Airplane
Barbara
Boat
Goldhill
Lenna
Mandrill
Milkdrop
Sailboat
